In [1]:
# do not forget to swapoff -a
import numpy as np       # linear algebra
import pylab as pl       # plots
import tensorflow as tf  # just for data sets (no deep learning yet)

In [2]:
# choose wisely
(train, label_train), (test, label_test) = tf.keras.datasets.mnist.load_data()
#(train, label_train), (test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

# make sure your data is floating point
test = test.astype(np.float32)
train = train.astype(np.float32)

# print shapes
print(train.shape, label_train.shape)
print(test.shape, label_test.shape)

((60000, 28, 28), (60000,))
((10000, 28, 28), (10000,))


In [3]:
# here we can subsample in the data set
num_test, num_train = 10000, 60000
test, label_test = test[:num_test], label_test[:num_test]
train, label_train = train[:num_train], label_train[:num_train]

# print shapes
print(train.shape, label_train.shape)
print(test.shape, label_test.shape)

((60000, 28, 28), (60000,))
((10000, 28, 28), (10000,))


In [4]:
def to_one_hot(labels, depths):
    
    result = np.zeros((labels.shape[0], depths))
    
    for index, label in enumerate(labels):
        result[index, label] = 1
    
    return result

In [5]:
label_train_1 = to_one_hot(label_train, 10)
label_test_1 = to_one_hot(label_test, 10)

print(label_train_1.shape)
print(label_test_1.shape)

(60000, 10)
(10000, 10)


In [6]:
# reinterpret R^{28 x 28} as R^{784}
train, test = train.reshape((-1, 784)), test.reshape((-1, 784))

# homogeneous embedding Ax + a <=> (A, a) (x) = (A x + a)
#                                  (0, 1) (1)   (   1   )
train_h = np.hstack((train, np.ones((train.shape[0], 1))))
test_h  = np.hstack((test, np.ones((test.shape[0], 1))))

In [7]:
# X A = Y   (m, 784+1) (785, 10) = (m, 10)
# obviously, we have to find X^+ such that X^+ X = id
A = np.linalg.pinv(train_h).dot(label_train_1)

print(A.shape)

(785, 10)


In [8]:
predicted_train = train_h.dot(A)
predicted_test  = test_h.dot(A)

In [9]:
print(np.mean(predicted_train.argmax(axis=1) == label_train))
print(np.mean(predicted_test.argmax(axis=1) == label_test))

0.8577333333333333
0.8603


In [10]:
def fit(train, label_train, test, label_test):
    """ train*A = label_train """
    
    A = np.linalg.pinv(train).dot(label_train)
  
    predicted_train = train.dot(A)
    predicted_test  = test.dot(A)
       
    return (np.mean(predicted_train.argmax(axis=1) == label_train.argmax(axis=1)),
            np.mean(predicted_test.argmax(axis=1)  == label_test.argmax(axis=1)))
    

In [11]:
for m in np.logspace(0.5, 1.0, num=16, base=train_h.shape[0]).astype(np.int64):
    print(m, fit(train_h[:m], label_train_1[:m], test_h, label_test_1))

(244, (1.0, 0.5634))
(353, (1.0, 0.4867))
(510, (1.0, 0.2364))
(736, (1.0, 0.4848))
(1062, (0.9971751412429378, 0.6493))
(1532, (0.9771540469973891, 0.7096))
(2211, (0.952962460425147, 0.7694))
(3191, (0.9272955186461924, 0.7999))
(4605, (0.9129207383279044, 0.8242))
(6645, (0.8991723100075244, 0.832))
(9589, (0.8819480654917092, 0.8354))
(13837, (0.8746115487461155, 0.8461))
(19968, (0.8664863782051282, 0.8529))
(28814, (0.8629832720205456, 0.8565))
(41579, (0.8593761273719907, 0.8603))
(60000, (0.8577333333333333, 0.8603))


In [12]:
# gradient descent
# L(A) = ||XA-Y||_F^2
# @L/@A = 2 Xt (XA-Y)

def fit(train, label_train, test, label_test, 
        num_iterations=2**16, alpha=1E-7, batch_size=32):
    
    A = np.zeros((train.shape[1], label_train.shape[1])) # (784+1, 10)
       
    for iteration in range(num_iterations):
        indices = np.random.choice(train.shape[0], batch_size, replace=False)
        
        X = train[indices]       # (m, 784+1)
        Y = label_train[indices] # (m, 10)
    
        G  = X.T.dot(X.dot(A)-Y)/(batch_size)
        A -= alpha*G
      
    predicted_train = train.dot(A)
    predicted_test  = test.dot(A)
       
    return (np.mean(predicted_train.argmax(axis=1) == label_train.argmax(axis=1)),
            np.mean(predicted_test.argmax(axis=1)  == label_test.argmax(axis=1)))



In [ ]:
for m in np.logspace(0.5, 1.0, num=16, base=train_h.shape[0]).astype(np.int64):    
    print(m, fit(train_h[:m], label_train_1[:m], test_h, label_test_1))

(244, (1.0, 0.6089))
(353, (1.0, 0.6526))
(510, (1.0, 0.6834))
(736, (0.998641304347826, 0.7181))
(1062, (0.9868173258003766, 0.7518))
(1532, (0.9484334203655352, 0.7774))
(2211, (0.9262777023971054, 0.8037))
(3191, (0.902851770604826, 0.8162))
(4605, (0.8964169381107492, 0.8208))
(6645, (0.8823175319789315, 0.8312))
(9589, (0.8673480029200125, 0.8341))
(13837, (0.861241598612416, 0.8406))
(19968, (0.8608774038461539, 0.8526))
(28814, (0.8574304157701117, 0.8555))
